Task: Data extraction through API (data source: Electrica De Espana)
Source: https://www.ree.es/es/apidatos

In [1]:
import pandas as pd
import json
import requests

In [154]:
def co2_evolution(start_date, end_date):
    #Enter the start_date and end_date in the following format: YYYY-MM-DDTHH:MM

    # API get request
    response = requests.get(f"https://apidatos.ree.es/en/datos/generacion/evolucion-estructura-generacion-emisiones-asociadas?start_date={start_date}&end_date={end_date}&time_trunc=day")
    response_json = response.json()
    data = response_json
    
    # Accessing data for "non-renewable type (Con emisiones de CO2 eq.)" ("id": 1)
    data1 = data["included"][0]["attributes"]["values"]
    evolution_co2_1 = pd.DataFrame(data1).drop("percentage", axis=1)
    evolution_co2_1["Type_id"] = 1
    evolution_co2_1["Description"] = "CO2 emission eq."


    # Accessing data for "renewable type (Sin emisiones de CO2 eq.)" ("id": 2)
    data2 = data["included"][1]["attributes"]["values"]
    evolution_co2_2 = pd.DataFrame(data2).drop("percentage", axis=1)
    evolution_co2_2["Type_id"] = 2
    evolution_co2_2["Description"] = "No CO2 emission eq."

    # Concat dataframes
    frames = [evolution_co2_1, evolution_co2_2]
    evolution_co2 = pd.concat(frames).reset_index(drop=True)
    evolution_co2['datetime'] = pd.to_datetime(evolution_co2["datetime"],format='%Y-%m')
    evolution_co2 = evolution_co2.rename(columns={'value': 'Value', "datetime":"Date"})
    return evolution_co2

In [155]:
# Get the data for 2020 and 2019
co2_20 = co2_evolution("2020-01-01T00:00", "2020-06-15T00:00")
co2_19 = co2_evolution("2019-01-01T00:00", "2019-06-15T00:00")

In [156]:
# Export the data in a .csv format
co2_20 = co2_20.to_csv("CO2-Evolution_2020.csv", index=False)
co2_19 = co2_19.to_csv("CO2-Evolution_2019.csv", index=False)